In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
import logging
from collections import Counter
import datetime
import os
import yaml

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
%run ../src/slope.py

In [3]:
OUTPUT_FOLDER = '../data/test_final/'
EPSG = CRS.WGS84
GRID_SIZE_X = 1
GRID_SIZE_Y = 1

IMAGE_X = 14*GRID_SIZE_X
IMAGE_Y = 14*GRID_SIZE_Y

TEST_X = 150
TEST_Y = 100 

with open("../config.yaml", 'r') as stream:
    key = (yaml.safe_load(stream))
    API_KEY = key['key']

In [4]:
# setup function to reproject coordinates
def convertCoords(xy, src='', targ=''):

    srcproj = osr.SpatialReference()
    srcproj.ImportFromEPSG(src)
    targproj = osr.SpatialReference()
    if isinstance(targ, str):
        targproj.ImportFromProj4(targ)
    else:
        targproj.ImportFromEPSG(targ)
    transform = osr.CoordinateTransformation(srcproj, targproj)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(xy[0], xy[1])
    pt.Transform(transform)
    return([pt.GetX(), pt.GetY()])

def bounding_box(point, x_offset_max = 140, y_offset_max = 140, expansion = 160):
    # LONG, LAT FOR SOME REASON
    tl = point
    
    if 48 <= tl[0] <= 54:
        epsg = 32639 if tl[1] > 0 else 32739
    if 42 <= tl[0] <= 48:
        epsg = 32638 if tl[1] > 0 else 32738
    if 36 <= tl[0] <= 42:
        epsg = 32637 if tl[1] > 0 else 32737
    if 30 <= tl[0] <= 36:
        epsg = 32636 if tl[1] > 0 else 32736
    if 24 <= tl[0] <= 30:
        epsg = 32635 if tl[1] > 0 else 32735
    if 18 <= tl[0] <= 24:
        epsg = 32634 if tl[1] > 0 else 32734

    tl = convertCoords(tl, 4326, epsg)
    
    br = (tl[0] + x_offset_max, tl[1] + y_offset_max)
    tl = ((tl[0] + (x_offset_max - 125 )), (tl[1] + (y_offset_max - 125*GRID_SIZE_Y)))
    distance1 = tl[0] - br[0]
    distance2 = tl[1] - br[1]
    print(distance1, distance2)
    EXPANSION = (expansion - np.mean([distance1, distance2]))/2 # should this be 155 or 160?
    
    br = [a - EXPANSION for a in br]
    tl = [a + EXPANSION for a in tl]
    
    after = [b - a for a,b in zip(br, tl)]
    #br = (br[0] + 20, br[1] + 20)
    #tl = (tl[0] - 20, tl[1] - 20)
    br = convertCoords(br, epsg, 4326)
    tl = convertCoords(tl, epsg, 4326)
    
    min_x = tl[0] # original X offset - 10 meters
    max_x = br[0] # original X offset + 10*GRID_SIZE meters
    
    min_y = tl[1] # original Y offset - 10 meters
    max_y = br[1] # original Y offset + 10 meters + 140 meters
    # (min_x, min_y), (max_x, max_y)
    # (bl, tr)
    return [(min_x, min_y), (max_x, max_y)]


def modify_bbox(point, expansion = 10):
    # LONG, LAT FOR SOME REASON
    min_x = point[0][0]
    max_x = point[1][0]
    min_y = point[0][1]
    max_y = point[1][1]
    
    if 48 <= min_x <= 54:
        epsg = 32639 if min_y > 0 else 32739
    if 42 <= min_x <= 48:
        epsg = 32638 if min_y > 0 else 32738
    if 36 <= min_x <= 42:
        epsg = 32637 if min_y > 0 else 32737
    if 30 <= min_x <= 36:
        epsg = 32636 if min_y > 0 else 32736
    if 24 <= min_x <= 30:
        epsg = 32635 if min_y > 0 else 32735
    if 18 <= min_x <= 24:
        epsg = 32634 if min_y > 0 else 32734
        
    bl = convertCoords((min_x, min_y), 4326, epsg)
    tr = convertCoords((max_x, max_y), 4326, epsg)
    
    bl = [a + expansion for a in bl]
    tr = [a - expansion for a in tr]
    after = [b - a for a,b in zip(tr, bl)]
    
    bl = convertCoords(bl, epsg, 4326)
    tr = convertCoords(tr, epsg, 4326)
    
    max_x = tr[0] 
    max_x = bl[0]
    
    min_y = bl[1] # original Y offset - 10 meters
    max_y = tr[1] # original Y offset + 10 meters + 140 meters
    # (min_x, min_y), (max_x, max_y)
    # (bl, tr)
    return [(min_x, min_y), (max_x, max_y)]

In [ ]:
# Test 1 == 13.727559, 38.316348
# Test 1.1 == , 38.246539, 13.934339
# Test 1.2 village with trees 39.532886, 12.870031
# Test 1.3 crop, trees 13.685493, 38.175899
# Test 1.4 trees on grass 13.581438, 38.171402
# Trees on farm 13.577898, 38.174085
# Segmented road 14.165738, 38.156894

# NEW 13.898518, 39.063205
# Uganda trees on farms 2.123535, 33.679550
# Rwanda marsh -2.059023, 29.981993

# 8 x 9 grid 14.190815, 38.151069

# 16 x 18 grid 14.245881, 38.173931
# 16 x 18 grid 2 14.318609, 38.141159
# 16 x 9 grid 14.346171, 38.167957
# test_tigray2 14.256921, 37.601867
# test_kenya 2.789021, 37.215458
# test_tigray3 13.567941, 38.157342

# landscape BL 13.567941, 38.157342
# landscape BR 13.567941, 38.25049890217167

coords = (13.567941, 38.157342)
coords = (coords[1], coords[0])
print(coords)
# Test 2 -- village 38.33024, 13.715228
# Test 3 -- farm 13.487358, 39.095307
#initial_point = bounding_box((39.095307, 13.487358))
initial_point = bounding_box(coords)
print(initial_point)

(38.157342, 13.567941)
-125.0 -125.0
[(38.15879260687674, 13.569369919322211), (38.157318974633895, 13.567918318679792)]


In [ ]:
import itertools
test = []

offsets_x = [x for x in range(140*GRID_SIZE_X, 140*(TEST_X + 1), 140*GRID_SIZE_X)]
offsets_y = [x for x in range(140*GRID_SIZE_Y, 140*(TEST_Y + 1), 140*GRID_SIZE_Y)]
perms = [(y, x) for x, y in itertools.product(offsets_y, offsets_x)]

for i in perms:
    bbx = bounding_box(initial_point[0], y_offset_max = i[1], x_offset_max = i[0])
    test.append(bbx)

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

...

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0 -125.0
-125.0

In [ ]:
min_x = []
max_x = []
min_y = []
max_y = []
for x in test:
    min_x.append(min((x[0][0], x[1][0])))
    max_x.append(max(x[0][0], x[1][0]))
    min_y.append(min(x[0][1], x[1][1]))
    max_y.append(max(x[0][1], x[1][1]))
    
print("West {} South {} East {} North {}".format(min(min_x), min(min_y), max(max_x), max(max_y)))

West 38.15832605584397 South 13.569347238122344 East 38.35303923647131 North 13.696690823232128


In [ ]:
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

def identify_clouds(bbox, epsg = EPSG, time = ('2018-01-01', '2018-12-31')):
    try:
        box = BBox(bbox, crs = epsg)
        cloud_request = WmsRequest(
            layer='CLOUD_DETECTION',
            bbox=box,
            time=time,
            width=(14*GRID_SIZE_X) + 2,
            height=16,
            image_format = MimeType.TIFF_d32f,
            maxcc=0.4,
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
            time_difference=datetime.timedelta(hours=24),
        )
        
        cloud_img = cloud_request.get_data()
        cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(cloud_img))
        means = np.mean(cloud_probs, (1, 2))
        cloud_steps = [i for i, val in enumerate(means) if val > 0.20]
        return cloud_steps, means
    except Exception as e:
        logging.fatal(e, exc_info=True)
    
    
def download_dem(bbox, epsg = EPSG):
    bbox = modify_bbox(bbox, expansion = 10)
    box = BBox(bbox, crs = epsg)
    dem_request = WmsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=box,
                         width=18,
                         height=18,
                         instance_id=API_KEY,
                         image_format=MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image = dem_request.get_data()[0]
    dem_image = calcSlope(dem_image.reshape((1, 18, 18)),
                  np.full((18, 18), 10), np.full((18, 18), 10), zScale = 1, minSlope = 0.02)
    dem_image = dem_image.reshape((18, 18, 1))
    dem_image = dem_image[1:17, 1:17, :]
    print(dem_image.shape)
    return dem_image #/ np.max(dem_image)


        
    
    
def download_tiles(bbox, epsg = EPSG, time = ('2018-01-01', '2018-12-31')):
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WmsRequest(
                layer='ALL_BANDS_NDVI',
                bbox=box,
                time=time,
                width=(14*GRID_SIZE_X) + 2,
                height=16,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.4,
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
                time_difference=datetime.timedelta(hours=24),
            )
        img_bands = image_request.get_data()
        return img_bands, image_request

    except Exception as e:
        logging.fatal(e, exc_info=True)
    

def calculate_and_save_best_images(cloud_steps, img_bands, image_request, means):
    begining_length = len(img_bands)
    clean_steps = np.array([x for x in range(len(img_bands)) if x not in cloud_steps])
    clean_means = np.array([val for x, val in enumerate(means) if x not in cloud_steps])
    keep_steps = []
    month_steps = []
    month_date = []
    month_hash = []
    for date in image_request.get_dates():
        month_steps.append(date.month)
        month_date.append(date.day)
    
    months = {}    
    for i in range(1, 13):
        month_i = [x for x, val in enumerate(month_steps) if val == i]
        month_clean_steps = [x for x in month_i if means[x] < 0.25]
        month_clean_dates = [val for x, val in enumerate(month_date) if x in month_clean_steps]
        month_first_half = [val for x, val in enumerate(month_clean_steps) if month_clean_dates[x] < 15]
        month_second_half = [val for x, val in enumerate(month_clean_steps) if month_clean_dates[x] >= 15]
        months[i] = {"first": month_first_half, "last": month_second_half}
            
    months_steps = {}  
    for i in months.keys():
        prior = []
        nxt = []
        first = months[i]["first"]
        second = months[i]["last"]
        if i < 12:
            nxt = months[i + 1]["first"]
        if i > 1: 
            prior = months[i - 1]["last"]
        if nxt:
            if prior:
                if not first and second:
                    months[i]["first"] = prior + second
                if not first and not second:
                    months[i]["first"] = prior + nxt
                if not second and first:
                    months[i]["last"] = first + nxt
                if not second and not first:
                    months[i]["last"] = prior + nxt
                    
    for i in months.keys():
        first = months[i]["first"]
        second = months[i]["last"]
        if not first and second:
            months[i]['first'] = months[i]['last']
        if not second and first:
            months[i]['last'] = months[i]['first']
            
    for i in months.keys():
        first = months[i]["first"]
        second = months[i]["last"]
        if not first:
            months[i]['first'] = months[i - 1]['last']
        if not second:
            months[i]['last'] = months[i + 1]['first']
            
    for i in months.keys():
        first = months[i]["first"]
        second = months[i]["last"]
        if not first:
            months[i]['first'] = months[i]['last']
        if not second:
            months[i]['last'] = months[i]['first']
            
    for i in months.keys():
        first = months[i]["first"]
        second = months[i]["last"]
        if not first:
            print("ERROR!!")
        if not second:
            print("ERROR")
    
    for i in months.keys():
        month_first = months[i]['first']
        if len(month_first) > 1:
            month_first = np.mean([val for x, val in enumerate(img_bands) if x in month_first], axis = 0)
        else:
            month_first = img_bands[month_first[0]]
        month_last = months[i]['last']
        if len(month_last) > 1:
            month_last = np.mean([val for x, val in enumerate(img_bands) if x in month_last], axis = 0)
        else:
            month_last = img_bands[month_last[0]]
        keep_steps.append(month_first)
        keep_steps.append(month_last)
    npify = np.stack(keep_steps)
    return(npify)

In [ ]:
np.min([x[0][1] for x in test])

13.570798831061708

In [ ]:
existing = [int(x[:-4]) for x in os.listdir("../data/test_final/") if '.npy' in x]
to_download = [x for x in range(0, len(test)) if x not in existing]
#print(to_download)
#existing = [x for x in range(0, 6994)]

In [ ]:
#errors = [] # 755
#

from sentinelhub import DataSource
from sentinelhub import CustomUrlParam




print("There are {} files to download".format(len(to_download)))
#for i in range(0, len(test)):
for i in to_download:
    print(i, perms[i], "row " + str(perms[i][1]/140), "column " + str(perms[i][0]/140))
    try:
        # Initiate hash tables
        cloud, means = identify_clouds(test[i])
        dem = download_dem(test[i])
        img, image_request = download_tiles(test[i])
        tiles = calculate_and_save_best_images(cloud, img, image_request, means) # 22, 16, 16, 10
        dem = np.tile(dem.reshape((1, 16, 16, 1)), (tiles.shape[0], 1, 1, 1))
        #dem = np.stack([dem] * tiles.shape[0]).reshape((tiles.shape[0], 16, 16, 1))
        tiles = np.concatenate([tiles, dem], axis = -1)
        np.save(OUTPUT_FOLDER + str(i*GRID_SIZE_X), tiles)
        #np.save(OUTPUT_FOLDER + str((i*2)+1), two)
        #np.save(OUTPUT_FOLDER + str((i*4)+2), three)
        #np.save(OUTPUT_FOLDER + str((i*4)+3), four)

    except Exception as e:
        logging.fatal(e, exc_info=True)
        errors.append(i)
        #continue

There are 8375 files to download
6625 (3640, 6300) row 45.0 column 26.0
(16, 16, 1)
6626 (3780, 6300) row 45.0 column 27.0
(16, 16, 1)
6627 (3920, 6300) row 45.0 column 28.0
(16, 16, 1)
6628 (4060, 6300) row 45.0 column 29.0
(16, 16, 1)
6629 (4200, 6300) row 45.0 column 30.0
(16, 16, 1)
6630 (4340, 6300) row 45.0 column 31.0
(16, 16, 1)
6631 (4480, 6300) row 45.0 column 32.0
(16, 16, 1)
6632 (4620, 6300) row 45.0 column 33.0
(16, 16, 1)
6633 (4760, 6300) row 45.0 column 34.0
(16, 16, 1)
6634 (4900, 6300) row 45.0 column 35.0
(16, 16, 1)
6635 (5040, 6300) row 45.0 column 36.0
(16, 16, 1)
6636 (5180, 6300) row 45.0 column 37.0
(16, 16, 1)
6637 (5320, 6300) row 45.0 column 38.0
(16, 16, 1)
6638 (5460, 6300) row 45.0 column 39.0
(16, 16, 1)
6639 (5600, 6300) row 45.0 column 40.0
(16, 16, 1)
6640 (5740, 6300) row 45.0 column 41.0
(16, 16, 1)
6641 (5880, 6300) row 45.0 column 42.0
(16, 16, 1)
6642 (6020, 6300) row 45.0 column 43.0
(16, 16, 1)
6643 (6160, 6300) row 45.0 column 44.0
(16, 16, 1

(16, 16, 1)
6783 (4760, 6440) row 46.0 column 34.0
(16, 16, 1)
6784 (4900, 6440) row 46.0 column 35.0
(16, 16, 1)
6785 (5040, 6440) row 46.0 column 36.0
(16, 16, 1)
6786 (5180, 6440) row 46.0 column 37.0
(16, 16, 1)
6787 (5320, 6440) row 46.0 column 38.0
(16, 16, 1)
6788 (5460, 6440) row 46.0 column 39.0
(16, 16, 1)
6789 (5600, 6440) row 46.0 column 40.0
(16, 16, 1)
6790 (5740, 6440) row 46.0 column 41.0
(16, 16, 1)
6791 (5880, 6440) row 46.0 column 42.0
(16, 16, 1)
6792 (6020, 6440) row 46.0 column 43.0
(16, 16, 1)
6793 (6160, 6440) row 46.0 column 44.0
(16, 16, 1)
6794 (6300, 6440) row 46.0 column 45.0
(16, 16, 1)
6795 (6440, 6440) row 46.0 column 46.0
(16, 16, 1)
6796 (6580, 6440) row 46.0 column 47.0
(16, 16, 1)
6797 (6720, 6440) row 46.0 column 48.0
(16, 16, 1)
6798 (6860, 6440) row 46.0 column 49.0
(16, 16, 1)
6799 (7000, 6440) row 46.0 column 50.0
(16, 16, 1)
6800 (7140, 6440) row 46.0 column 51.0
(16, 16, 1)
6801 (7280, 6440) row 46.0 column 52.0
(16, 16, 1)
6802 (7420, 6440) r

(16, 16, 1)
6942 (6020, 6580) row 47.0 column 43.0
(16, 16, 1)
6943 (6160, 6580) row 47.0 column 44.0
(16, 16, 1)
6944 (6300, 6580) row 47.0 column 45.0
(16, 16, 1)
6945 (6440, 6580) row 47.0 column 46.0
(16, 16, 1)
6946 (6580, 6580) row 47.0 column 47.0
(16, 16, 1)
6947 (6720, 6580) row 47.0 column 48.0
(16, 16, 1)
6948 (6860, 6580) row 47.0 column 49.0
(16, 16, 1)
6949 (7000, 6580) row 47.0 column 50.0
(16, 16, 1)
6950 (7140, 6580) row 47.0 column 51.0
(16, 16, 1)
6951 (7280, 6580) row 47.0 column 52.0
(16, 16, 1)
6952 (7420, 6580) row 47.0 column 53.0
(16, 16, 1)
6953 (7560, 6580) row 47.0 column 54.0
(16, 16, 1)
6954 (7700, 6580) row 47.0 column 55.0
(16, 16, 1)
6955 (7840, 6580) row 47.0 column 56.0
(16, 16, 1)
6956 (7980, 6580) row 47.0 column 57.0
(16, 16, 1)
6957 (8120, 6580) row 47.0 column 58.0
(16, 16, 1)
6958 (8260, 6580) row 47.0 column 59.0
(16, 16, 1)
6959 (8400, 6580) row 47.0 column 60.0
(16, 16, 1)
6960 (8540, 6580) row 47.0 column 61.0
(16, 16, 1)
6961 (8680, 6580) r

(16, 16, 1)
7101 (7280, 6720) row 48.0 column 52.0
(16, 16, 1)
7102 (7420, 6720) row 48.0 column 53.0
(16, 16, 1)
7103 (7560, 6720) row 48.0 column 54.0
(16, 16, 1)
7104 (7700, 6720) row 48.0 column 55.0
(16, 16, 1)
7105 (7840, 6720) row 48.0 column 56.0
(16, 16, 1)
7106 (7980, 6720) row 48.0 column 57.0
(16, 16, 1)
7107 (8120, 6720) row 48.0 column 58.0
(16, 16, 1)
7108 (8260, 6720) row 48.0 column 59.0
(16, 16, 1)
7109 (8400, 6720) row 48.0 column 60.0
(16, 16, 1)
7110 (8540, 6720) row 48.0 column 61.0
(16, 16, 1)
7111 (8680, 6720) row 48.0 column 62.0
(16, 16, 1)
7112 (8820, 6720) row 48.0 column 63.0
(16, 16, 1)
7113 (8960, 6720) row 48.0 column 64.0
(16, 16, 1)
7114 (9100, 6720) row 48.0 column 65.0
(16, 16, 1)
7115 (9240, 6720) row 48.0 column 66.0
(16, 16, 1)
7116 (9380, 6720) row 48.0 column 67.0
(16, 16, 1)
7117 (9520, 6720) row 48.0 column 68.0
(16, 16, 1)
7118 (9660, 6720) row 48.0 column 69.0
(16, 16, 1)
7119 (9800, 6720) row 48.0 column 70.0
(16, 16, 1)
7120 (9940, 6720) r

(16, 16, 1)
7260 (8540, 6860) row 49.0 column 61.0
(16, 16, 1)
7261 (8680, 6860) row 49.0 column 62.0
(16, 16, 1)
7262 (8820, 6860) row 49.0 column 63.0
(16, 16, 1)
7263 (8960, 6860) row 49.0 column 64.0
(16, 16, 1)
7264 (9100, 6860) row 49.0 column 65.0
(16, 16, 1)
7265 (9240, 6860) row 49.0 column 66.0
(16, 16, 1)
7266 (9380, 6860) row 49.0 column 67.0
(16, 16, 1)
7267 (9520, 6860) row 49.0 column 68.0
(16, 16, 1)
7268 (9660, 6860) row 49.0 column 69.0
(16, 16, 1)
7269 (9800, 6860) row 49.0 column 70.0
(16, 16, 1)
7270 (9940, 6860) row 49.0 column 71.0
(16, 16, 1)
7271 (10080, 6860) row 49.0 column 72.0
(16, 16, 1)
7272 (10220, 6860) row 49.0 column 73.0
(16, 16, 1)
7273 (10360, 6860) row 49.0 column 74.0
(16, 16, 1)
7274 (10500, 6860) row 49.0 column 75.0
(16, 16, 1)
7275 (10640, 6860) row 49.0 column 76.0
(16, 16, 1)
7276 (10780, 6860) row 49.0 column 77.0
(16, 16, 1)
7277 (10920, 6860) row 49.0 column 78.0
(16, 16, 1)
7278 (11060, 6860) row 49.0 column 79.0
(16, 16, 1)
7279 (11200

In [ ]:
errors